# Code Generator
Using a Frontier model to generate high performance C++ code from Python code

In [ ]:
# Imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [ ]:
# Environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [ ]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"


In [ ]:
# Prompt sent to the LLM that powers our tool
# It tells the model *exactly* how to behave when translating Python ➜ C++

system_message = (
    # 1️⃣ Assign the model a role: Windows-focused Python-to-C++ translator
    "You are an assistant specialized in translating Python code into high-performance C++ optimized for Windows systems. "
    # 2️⃣ Require pure C++ output—no prose, so the response can drop straight into a .cpp file
    "Your response must consist solely of C++ code, producing output identical to the original Python version. "
    # 3️⃣ Push for maximum speed: keep comments sparse and purposeful
    "Prioritize execution speed and efficiency, using minimal and purposeful comments. Do not include any explanations or non-code content."
)


In [ ]:

'''
Build a single, well-structured prompt that instructs an LLM to translate
arbitrary Python snippets into highly optimized, Windows-friendly C++.
The function appends the user’s Python code to a fixed instruction header
demanding identical output, strict type handling, minimal comments, and
the necessary standard headers, then returns the complete prompt string.
'''

def user_prompt_for(python):
    user_prompt = (
        "Convert the following Python code into high-performance C++ optimized for Windows. "
        "Ensure the C++ implementation produces identical output with maximum execution speed. "
        "Respond only with C++ code, using minimal but meaningful comments where necessary. "
        "Be precise with numeric types to prevent integer overflows. "
        "Include all required standard headers, such as <iostream> and <iomanip>, to ensure the code compiles cleanly.\n\n"
    )
    user_prompt += python
    return user_prompt

In [ ]:
'''
Compose the message list expected by the chat API:
- System message sets the model’s role/constraints.
- User message supplies the generated prompt that embeds the
  Python code to be translated into high-performance C++.
'''

def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [ ]:
# Write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [ ]:
'''
Send the constructed messages to the OpenAI chat API, stream the model’s
C++ response in real time, echo each fragment to stdout, and finally save
the full reply to disk with write_output().
'''

def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [ ]:
'''
Send the Python snippet to Claude, stream back its C++ translation
in real time, echo each fragment to the console, then persist the
full response to disk via write_output().
'''

def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [ ]:
# Creating python code to be looked at by models

pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
exec(pi)

Result: 3.141592658589
Execution Time: 7.927061 seconds


In [ ]:
# GPT output of C++

optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

// Function to perform the calculation
double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        double j1 = static_cast<double>(i * param1 - param2);
        result -= 1.0 / j1;
        double j2 = static_cast<double>(i * param1 + param2);
        result += 1.0 / j2;
    }
    return result;
}

int main() {
    int iterations = 100000000;
    int param1 = 4;
    int param2 = 1;

    // Timing the execution
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(iterations, param1, param2) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    // Calculating execution time
    std::chrono::duration<double> elapsed_seconds = end_time - start_time;

    // Output results
    std::cout << "Result: " << std::setprecision(12) << result << std::endl;
    std::cout << "Execution Time: " << std::f

In [15]:
exec(pi)

Result: 3.141592658589
Execution Time: 7.830400 seconds


# Compiling C++ and executing

This next cell contains the command to compile a C++ file on my M1 Mac.  
It compiles the file `optimized.cpp` into an executable called `optimized`  
Then it runs the program called `optimized`

In [ ]:
# Compile C++ and run the executable

!g++ -O3 -std=c++17 optimized.cpp -o optimized.exe
!optimized.exe

In [ ]:
# Claude output of C++

optimize_claude(pi)

#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int64_t iterations, int64_t param1, int64_t param2) {
    double result = 1.0;
    for (int64_t i = 1; i <= iterations; ++i) {
        int64_t j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    
    double result = calculate(100'000'000, 4, 1) * 4;
    
    auto end_time = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << duration.count() / 1e6 << " seconds" << std::endl;

    return 0;
}

In [ ]:
# Repeat for Claude - again, use the right approach for your platform

!g++ -O3 -std=c++17 optimized.cpp -o optimized.exe
!optimized.exe

In [17]:
exec(pi)

Result: 3.141592658589
Execution Time: 10.134040 seconds


In [ ]:
# Enhancing python code to be read by models

python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [19]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 25.417762 seconds


In [ ]:
# GPT output of C++

optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <limits>
#include <chrono>

uint32_t lcg(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 1u << 31) {
    return (a * seed + c) % m;
}

int64_t max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    std::vector<int> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        seed = lcg(seed);
        random_numbers[i] = seed % (max_val - min_val + 1) + min_val;
    }

    int64_t max_sum = std::numeric_limits<int64_t>::min();
    for (int i = 0; i < n; ++i) {
        int64_t current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    return max_sum;
}

int64_t total_max_subarray_sum(int n, uint32_t initial_seed, int min_val, int max_val) {
    int64_t total_sum = 0;
    uint32_t seed = initial_seed;
    for (int i = 0; i < 20; ++i) {
    

In [ ]:
# Compile C++ and run the executable

!g++ -O3 -std=c++17 optimized.cpp -o optimized.exe
!optimized.exe

In [ ]:
# Claude output of C++

optimize_claude(python_hard)

#include <iostream>
#include <vector>
#include <limits>
#include <chrono>
#include <iomanip>

using namespace std;
using namespace chrono;

class LCG {
private:
    uint64_t value;
    const uint64_t a = 1664525;
    const uint64_t c = 1013904223;
    const uint64_t m = 1ULL << 32;

public:
    LCG(uint64_t seed) : value(seed) {}

    uint64_t next() {
        value = (a * value + c) % m;
        return value;
    }
};

int64_t max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
    LCG lcg_gen(seed);
    vector<int64_t> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg_gen.next() % (max_val - min_val + 1) + min_val;
    }

    int64_t max_sum = numeric_limits<int64_t>::min();
    for (int i = 0; i < n; ++i) {
        int64_t current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            max_sum = max(max_sum, current_sum);
        }
    }
    return max_sum;
}

int64_t total_max_subarra

In [ ]:
# Compile C++ and run the executable

!g++ -O3 -std=c++17 optimized.cpp -o optimized.exe
!optimized.exe

In [ ]:
'''
Stream GPT output as incremental chunks: we send the prepared message list
to the OpenAI chat endpoint, collect each delta fragment, build the running
response, strip Markdown code fences (` ```cpp ` and trailing backticks),
and yield the clean C++ text so callers (e.g., a Gradio UI) can display it
live while the model is still generating.
'''

def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
'''
Stream Claude’s response in real time: we call the Anthropic chat endpoint
with our system directive and user-specific prompt, then iterate over the
text stream, building the cumulative reply. Markdown code fences
(` ```cpp ... ``` `) are stripped on the fly, and each updated snapshot is
yielded so a UI (e.g., Gradio) can render the growing C++ translation live.
'''

def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
'''
Dispatch to the proper streaming generator based on the target model
(GPT or Claude). The chosen generator yields incremental C++ output;
this wrapper forwards each partial string so the caller can stream the
translation live. Raises ValueError for unsupported model names.
'''

def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far      

In [ ]:
'''
Gradio UI: paste Python, pick an LLM (GPT or Claude), and stream back the
high-performance C++ translation. The first textbox holds the source Python
snippet, the second displays the generated C++, a dropdown selects the model,
and the “Convert code” button launches the optimization pipeline. Finally,
the interface opens in the browser for interactive use.
'''

with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
'''
Execute an arbitrary Python snippet and return everything it prints.
The function redirects sys.stdout to an in-memory StringIO buffer,
runs the code via exec(), then restores the original stdout in a
finally block to ensure cleanup—even if the user code raises.
'''

def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [ ]:
'''
Compile and execute C++ code on Windows using MinGW’s g++:
1. Save the incoming C++ string to optimized.cpp via write_output().
2. Invoke g++ with -Ofast and C++17, emitting optimized.exe.
3. If compilation succeeds, run the binary and capture its stdout.
4. If either step fails, return the compiler or runtime stderr so the
   caller sees the exact error message.
'''

import subprocess

def execute_cpp(code):
    write_output(code)  # writes optimized.cpp

    try:
        compile_cmd = [
            "g++",
            "-Ofast",
            "-std=c++17",
            "optimized.cpp",
            "-o",
            "optimized.exe"
        ]
        compile_result = subprocess.run(
            compile_cmd,
            check=True,
            text=True,
            capture_output=True
        )

        run_cmd = ["optimized.exe"]
        run_result = subprocess.run(
            run_cmd,
            check=True,
            text=True,
            capture_output=True
        )
        return run_result.stdout

    except subprocess.CalledProcessError as e:
        return e.stderr


In [ ]:
'''
CSS theme: give Python code blocks a Python-blue background (#306998)
and C++ blocks a dark-green background (#050).
'''

css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [ ]:
'''
Gradio interface for live Python ➜ C++ conversion and execution.

Layout
• Markdown header for context.
• First row: side-by-side textboxes—editable Python (prefilled) and generated C++.
• Second row: model selector (GPT or Claude).
• Third row: “Convert code” triggers the LLM translation.
• Fourth row: “Run Python” and “Run C++” buttons execute code in-browser.
• Fifth row: color-themed text areas show stdout from each run.

Custom CSS (python / cpp classes) colors the result panes.
'''

with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)